# Desenvolver o codigo para automatizar a busca no Rec Aq

* https://www.reclameaqui.com.br/empresa/bet365/lista-reclamacoes/?pagina=1

In [162]:
from bs4 import BeautifulSoup
from unidecode import unidecode

import pandas as pd
import numpy as np

import string
import requests

In [231]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}

def get_complains(company_id, company_name):

    complains = {'descriptions': [], 'hrefs': []}
    
    for n in range(0, 1001, 10):
        url = f'https://iosearch.reclameaqui.com.br/raichu-io-site-search-v1/query/companyComplains/10/{n}'
        res = requests.get(url, params={'company': company_id}, headers=HEADERS).json()

        for x in range(10):
            data = res['complainResult']['complains']['data']
            
            description = data[x]['description']
            soup = BeautifulSoup(description, 'lxml').text
            complains['descriptions'].append(soup)
            
            id_ = data[x]['id']
            title = data[x]['title']
            clean_title = title.translate(str.maketrans('', '', string.punctuation))
            href = clean_title.lower() + '_' + id_
            href = unidecode(href.replace(' ', '-'))
            complains['hrefs'].append(href)
    
    company_complains = {company_name: complains}
    
    return company_complains